In [12]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, BaseMessage
from pydantic import BaseModel, Field
import json
import os
import subprocess
import dotenv

dotenv.load_dotenv()

model = ChatGoogleGenerativeAI(model="gemini-2.5-pro", )

os.makedirs("./temp", exist_ok=True)

def write_file(filename: str, content: str) -> str:
    """写入文件到temp目录"""
    try:
        filepath = os.path.join("./temp", filename)
        with open(filepath, "w", encoding="utf-8") as f:
            f.write(content)
        return f"文件已成功写入: {filepath}"
    except Exception as e:
        return f"写入文件失败: {str(e)}"

def run_python_file(filename: str) -> str:
    """运行temp目录下的Python文件"""
    try:
        filepath = os.path.join("./temp", filename)
        if not os.path.exists(filepath):
            return f"文件不存在: {filepath}"
        
        result = subprocess.run(
            ["python3", filepath], 
            capture_output=True, 
            text=True,
            cwd="./temp"
        )
        
        output = ""
        if result.stdout:
            output += f"输出:\n{result.stdout}\n"
        if result.stderr:
            output += f"错误:\n{result.stderr}\n"
        if result.returncode != 0:
            output += f"返回码: {result.returncode}\n"
            
        return output if output else "程序运行完成，无输出"
    except Exception as e:
        return f"运行Python文件失败: {str(e)}"

print("工具函数已定义完成")


工具函数已定义完成


In [15]:

from pydantic import field_validator
from typing import Union

class ToolCall(BaseModel):
    """工具调用"""
    tool_name: str = Field(description="要调用的工具名称")
    arguments: dict = Field(description="工具调用的参数")
    
    @field_validator('arguments', mode='before')
    @classmethod
    def parse_arguments(cls, v):
        """如果arguments是字符串，尝试解析为字典"""
        if isinstance(v, str):
            try:
                return json.loads(v)
            except json.JSONDecodeError:
                raise ValueError(f"无法解析arguments: {v}")
        return v

structured_model = model.with_structured_output(ToolCall)

system_prompt = f"""
你是一个编程助手。你可以使用以下工具：

write_file:
- 写入文件到temp目录
- arguments: filename, content

run_python_file:
- 运行temp目录下的Python文件
- arguments: filename

用户会给你一个任务，你需要决定调用哪个工具并提供相应的参数。
如果任务需要多个步骤，请先完成第一步。
"""

user_task = "请写一个Python代码来验证一维Normal分布的最大似然估计(MLE)公式，使用模拟数据进行验证"

messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=user_task)
]

tool_call = structured_model.invoke(messages)

if tool_call.tool_name == "write_file":  # type: ignore
    result = write_file(**tool_call.arguments)  # type: ignore
elif tool_call.tool_name == "run_python_file":  # type: ignore
    result = run_python_file(**tool_call.arguments)  # type: ignore
else:
    result = f"未知工具: {tool_call.tool_name}"  # type: ignore
print(result)

文件已成功写入: ./temp/mle_verification.py


In [ ]:
tool_call

In [ ]:
from langchain_core.tools import tool
from langchain_core.messages import ToolMessage

# 使用装饰器定义工具
@tool
def write_file_tool(filename: str, content: str) -> str:
    """写入文件到temp目录"""
    return write_file(filename, content)

@tool  
def run_python_file_tool(filename: str) -> str:
    """运行temp目录下的Python文件"""
    return run_python_file(filename)

# 创建带有工具的模型
tools = [write_file_tool, run_python_file_tool]
llm_with_tools = model.bind_tools(tools)

print("=== 方法二：使用 LangChain Function Calling ===")
print(f"用户任务: {user_task}")

# 创建agent循环
messages = [
    SystemMessage(content="你是一个编程助手。用户会给你编程任务，你需要使用可用的工具来完成任务。"),
    HumanMessage(content=user_task)
]

max_iterations = 5
current_iteration = 0

while current_iteration < max_iterations:
    current_iteration += 1
    print(f"\n--- 迭代 {current_iteration} ---")
    
    # 调用模型
    response = llm_with_tools.invoke(messages)
    messages.append(response)
    
    print(f"模型回复: {response.content}")
    
    # 检查是否有工具调用
    if hasattr(response, 'tool_calls') and response.tool_calls:  # type: ignore
        print(f"工具调用: {len(response.tool_calls)} 个")  # type: ignore
        
        # 执行所有工具调用
        for tool_call in response.tool_calls:  # type: ignore
            print(f"调用工具: {tool_call['name']} with args: {tool_call['args']}")
            
            # 执行工具
            if tool_call['name'] == 'write_file_tool':
                result = write_file_tool.invoke(tool_call['args'])
            elif tool_call['name'] == 'run_python_file_tool':
                result = run_python_file_tool.invoke(tool_call['args'])
            else:
                result = f"未知工具: {tool_call['name']}"
            
            print(f"工具结果: {result}")
            
            # 将工具结果添加到消息中
            messages.append(ToolMessage(
                content=result,
                tool_call_id=tool_call['id']
            ))
        
        # 如果有工具调用，继续下一轮
        continue
    else:
        # 没有工具调用，任务完成
        print("任务完成！")
        break

print(f"\n总共进行了 {current_iteration} 轮对话")
